In [2]:
import keras
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler

2024-04-19 13:39:19.245648: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 13:39:19.429518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 13:39:19.429567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 13:39:19.440098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-19 13:39:19.469473: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 13:39:19.469945: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
X, y = load_breast_cancer(return_X_y=True )
xtrainF, xtestF, ytrainF, ytestF = train_test_split(X, y, random_state=42 ,test_size=0.20)

## l2

In [4]:
from functools import partial
def Model(regularizer):
    
    #initalizaer = keras.initializers.VarianceScaling(scale=1., mode='fan_in', distribution='normal',seed=32)
    regularizedDense = partial(
        keras.layers.Dense,
        activation=keras.activations.selu, 
        kernel_initializer=keras.initializers.lecun_normal,
        kernel_regularizer=regularizer
    )
    model = keras.models.Sequential(
        layers = [
            keras.layers.BatchNormalization(),
            regularizedDense(30),
            keras.layers.BatchNormalization(),
            regularizedDense(15),
            keras.layers.BatchNormalization(),
            regularizedDense(10),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(1,activation=keras.activations.selu ,kernel_initializer=keras.initializers.lecun_normal)
        ]
    )
    return model

In [5]:
model = Model(keras.regularizers.l2(l2=0.01))
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9, nesterov=True, clipnorm=0.1),
    loss      = keras.losses.binary_crossentropy,
    metrics   = [keras.metrics.binary_accuracy]
)


model.fit( xtrainF, ytrainF, epochs=32, batch_size=32)


Epoch 1/32
15/15 [==============================] - 2s 3ms/step - loss: 8.5814 - binary_accuracy: 0.3385
Epoch 2/32
15/15 [==============================] - 0s 2ms/step - loss: 8.3222 - binary_accuracy: 0.3473
Epoch 3/32
15/15 [==============================] - 0s 2ms/step - loss: 7.4018 - binary_accuracy: 0.4044
Epoch 4/32
15/15 [==============================] - 0s 2ms/step - loss: 6.8729 - binary_accuracy: 0.4659
Epoch 5/32
15/15 [==============================] - 0s 2ms/step - loss: 5.8505 - binary_accuracy: 0.5560
Epoch 6/32
15/15 [==============================] - 0s 3ms/step - loss: 5.4587 - binary_accuracy: 0.5560
Epoch 7/32
15/15 [==============================] - 0s 3ms/step - loss: 4.7335 - binary_accuracy: 0.6176
Epoch 8/32
15/15 [==============================] - 0s 2ms/step - loss: 4.2658 - binary_accuracy: 0.6462
Epoch 9/32
15/15 [==============================] - 0s 2ms/step - loss: 4.1540 - binary_accuracy: 0.6835
Epoch 10/32
15/15 [==============================] - 0s

In [6]:
pred = model.predict(xtestF)>0.5

4/4 [==============================] - 0s 2ms/step


In [7]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        43
           1       0.97      1.00      0.99        71

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



## l1

In [8]:
model = Model(keras.regularizers.l1(l1=0.01))
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9, nesterov=True, clipnorm=0.1),
    loss      = keras.losses.binary_crossentropy,
    metrics   = [keras.metrics.binary_accuracy]
)


model.fit( xtrainF, ytrainF, epochs=32, batch_size=32)


Epoch 1/32
15/15 [==============================] - 2s 2ms/step - loss: 7.0591 - binary_accuracy: 0.5868
Epoch 2/32
15/15 [==============================] - 0s 2ms/step - loss: 6.7469 - binary_accuracy: 0.6220
Epoch 3/32
15/15 [==============================] - 0s 2ms/step - loss: 6.3028 - binary_accuracy: 0.6484
Epoch 4/32
15/15 [==============================] - 0s 3ms/step - loss: 6.0723 - binary_accuracy: 0.6549
Epoch 5/32
15/15 [==============================] - 0s 3ms/step - loss: 5.5540 - binary_accuracy: 0.6945
Epoch 6/32
15/15 [==============================] - 0s 2ms/step - loss: 5.1504 - binary_accuracy: 0.7077
Epoch 7/32
15/15 [==============================] - 0s 3ms/step - loss: 4.9489 - binary_accuracy: 0.7297
Epoch 8/32
15/15 [==============================] - 0s 3ms/step - loss: 4.5854 - binary_accuracy: 0.7429
Epoch 9/32
15/15 [==============================] - 0s 2ms/step - loss: 4.1040 - binary_accuracy: 0.7868
Epoch 10/32
15/15 [==============================] - 0s

In [9]:
pred = model.predict(xtestF)>0.5
print(classification_report(ytestF,pred))

4/4 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114



## Dropout

In [10]:
from functools import partial
def Model(regularizer):
    
    #initalizaer = keras.initializers.VarianceScaling(scale=1., mode='fan_in', distribution='normal',seed=32)
    regularizedDense = partial(
        keras.layers.Dense,
        activation=keras.activations.selu, 
        kernel_initializer=keras.initializers.lecun_normal,
    )
    model = keras.models.Sequential(
        layers = [
            keras.layers.BatchNormalization(),
            regularizedDense(30),
            keras.layers.BatchNormalization(),
            regularizedDense(15),
            keras.layers.Dropout(0.1),
            keras.layers.BatchNormalization(),
            regularizedDense(10),
            keras.layers.Dropout(0.1),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(1,activation=keras.activations.selu ,kernel_initializer=keras.initializers.lecun_normal)
        ]
    )
    return model

In [18]:
model = Model(keras.regularizers.l1(l1=0.01))
lr = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=32*len(xtrainF),decay_rate=0.1)

call = keras.callbacks.EarlyStopping(monitor='loss', patience=9)
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=lr,momentum=0.9, nesterov=True, clipnorm=0.1),
    loss      = keras.losses.binary_crossentropy,
    metrics   = [keras.metrics.binary_accuracy]
    
)

model.fit( xtrainF, ytrainF, epochs=32, batch_size=32, callbacks=[call])


Epoch 1/32
15/15 [==============================] - 2s 2ms/step - loss: 4.4734 - binary_accuracy: 0.5516
Epoch 2/32
15/15 [==============================] - 0s 2ms/step - loss: 1.8108 - binary_accuracy: 0.7626
Epoch 3/32
15/15 [==============================] - 0s 2ms/step - loss: 0.8152 - binary_accuracy: 0.8747
Epoch 4/32
15/15 [==============================] - 0s 2ms/step - loss: 0.7214 - binary_accuracy: 0.9143
Epoch 5/32
15/15 [==============================] - 0s 2ms/step - loss: 0.6447 - binary_accuracy: 0.9099
Epoch 6/32
15/15 [==============================] - 0s 2ms/step - loss: 0.6917 - binary_accuracy: 0.9275
Epoch 7/32
15/15 [==============================] - 0s 2ms/step - loss: 0.3362 - binary_accuracy: 0.9451
Epoch 8/32
15/15 [==============================] - 0s 3ms/step - loss: 0.2695 - binary_accuracy: 0.9495
Epoch 9/32
15/15 [==============================] - 0s 3ms/step - loss: 0.3386 - binary_accuracy: 0.9407
Epoch 10/32
15/15 [==============================] - 0s

In [19]:
pred = model.predict(xtestF)>0.5
print(classification_report(ytestF,pred))

4/4 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.98      0.92        43
           1       0.98      0.92      0.95        71

    accuracy                           0.94       114
   macro avg       0.93      0.95      0.94       114
weighted avg       0.94      0.94      0.94       114

